# Two stage miller OTA Sizing Script
Reference: 
- CMOS Analog Circuit Design, Phillip E. Allen
- gm/ID, Boris Murman

In [1]:
from pygmid import Lookup as lk
import numpy as np

In [2]:
#Specs
gain_bw = 10e6 #minimum gain bandwidth
fc = gain_bw
gain_db = 20 #minimum gain 20 dB
phase_margin = 90 #minimum phase margin 90 deg
c_load = 30e-12 # load capacitance
vdd = 3.3
vss = 0
vdsat_pmos = 0 #vds saturation source follower PMOS output stage
vdsat_nmos = 0 #vds saturation tail current source
v_swing = vdd + abs(vss) - vdsat_pmos - vdsat_nmos
ts = 5 / (2 * np.pi * gain_bw)
sr = v_swing / ts
c_miller = 0.22 * c_load #C Miller Compensation
i_tail = sr * c_miller #I tail
gm_1 = fc * 2 * np.pi * c_miller #input stage transconductace (times 3 for PVT margin)

print(f"Voltage Swing: {v_swing} V")
print(f"Settling Time: {round(ts*1e6,2)} us")
print(f"Slew Rate: {round(sr/1e6,2)} V/us")
print(f"C Miller: {round(c_miller*1e12,2)} pF")
print(f"Tail current: {round(i_tail*1e6,2)} uA")
print(f"gm 1: {round(gm_1*1000,2)} mS")

Voltage Swing: 3.3 V
Settling Time: 0.08 us
Slew Rate: 41.47 V/us
C Miller: 6.6 pF
Tail current: 273.7 uA
gm 1: 0.41 mS


In [3]:
def get_w(mos_type, gm, gm_id_spec, L_spec, vds, title='specs'):
    
    #Define the MOS type
    if mos_type == 0:
        mos_file = 'nfet_03v3.mat'
    elif mos_type == 1:
        mos_file = 'pfet_03v3.mat'
    else:
        print(f"MOS type: 0: NMOS 3.3V, 1: PMOS 3.3")
        return
    print(f"[Transistor: {title}]")
    print(f"gm: {round(gm*1e3,3)} mS")
    # the gm_gds we look up and calculate gds from it
    mos = lk(mos_file)
    vgs = mos.look_upVGS(GM_ID=gm_id_spec, L=L_spec, VDS=vds, VSB=0.0)
    vgs = mos.look_upVGS(GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0.0)
    print('V_GS =', round(float(vgs), 3), 'V')

    # the gm_gds we look up and calculate gds from it
    gm_gds = mos.lookup('GM_GDS', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
    gds = gm / gm_gds
    print('gds =', round(gds/1e-9, 2), 'nS')

    # find f_T (which is not stored directly, but we can find the gm to gate capacitance ratio)
    gm_cgg = mos.lookup('GM_CGG', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
    f_T = gm_cgg / (2*np.pi)
    print('Cgg =', round(gm/gm_cgg/1e-15, 1), 'fF')
    print('f_T =', round(f_T/1e6, 1), 'MHz')

    # find the W of the diode transistor
    id_spec = gm/gm_id_spec
    print('Id =', round(id_spec*1000000, 2), 'uA, rounded Id =', round(id_spec*2000000)/2, 'uA')
    id_w = mos.lookup('ID_W', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
    w = id_spec / id_w
    print('W =', round(w, 2), 'um, rounded W =', round(w*2)/2, 'um')      

    return [gds,id_spec]

In [4]:
#Transistor sizing
length =0.8   # L = 2 um
vds = vdd/2 
#Input Pairs
gmid_1 = 10
id_1 = gm_1/gmid_1
gds_1,id_1 = get_w(0,gm_1,gmid_1,length,vds,"M1 & M2")

#Active Load
gmid_2 = 5
gm_2 = gmid_2 * id_1
gds_2,id_2 = get_w(1,gm_2,gmid_2,length,vds,"M3 & M4")

#Tail Current
gmid_5 = 5
gm_5 = gmid_5 * id_1 * 2
gds_5,id_5 = get_w(0,gm_5,gmid_5,length,vds,"M5")

[Transistor: M1 & M2]
gm: 0.415 mS


V_GS = 0.837 V
gds = 1616.14 nS
Cgg = 37.5 fF
f_T = 1760.9 MHz
Id = 41.47 uA, rounded Id = 41.5 uA
W = 15.12 um, rounded W = 15.0 um
[Transistor: M3 & M4]
gm: 0.207 mS
V_GS = 1.149 V
gds = 1017.61 nS
Cgg = 42.9 fF
f_T = 769.1 MHz
Id = 41.47 uA, rounded Id = 41.5 uA
W = 14.6 um, rounded W = 14.5 um
[Transistor: M5]
gm: 0.415 mS
V_GS = 1.038 V
gds = 2130.35 nS
Cgg = 19.9 fF
f_T = 3309.9 MHz
Id = 82.94 uA, rounded Id = 83.0 uA
W = 7.3 um, rounded W = 7.5 um


In [5]:
#Common Source PMOS Output Stage
gmid_6 = 5
gm_6 = 10 * gm_1
gds_6,id_6 = get_w(1, gm_6, gmid_6, 0.5, vds, "M6")

[Transistor: M6]
gm: 4.147 mS
V_GS = 1.141 V
gds = 36269.83 nS
Cgg = 328.1 fF
f_T = 2011.5 MHz
Id = 829.38 uA, rounded Id = 829.5 uA
W = 167.08 um, rounded W = 167.0 um


In [6]:
#Current source NMOS Output Stage
gmid_7 = 5
id_7 = id_6
gm_7 = gmid_7 * id_7
gds_7,id_7 = get_w(0, gm_7, gmid_7, length, vds, "M7")

#Nulling Resistor
c_miller=0.8e-12
rz = (c_miller + c_load)/c_miller/gm_6
print(f"Rz: {rz}")

[Transistor: M7]
gm: 4.147 mS


V_GS = 1.038 V
gds = 21303.53 nS
Cgg = 199.4 fF
f_T = 3309.9 MHz
Id = 829.38 uA, rounded Id = 829.5 uA
W = 73.04 um, rounded W = 73.0 um
Rz: 9284.038347027228


In [7]:
#Open Loop Gain
gm_in = gm_1
gm_out = gm_6
gds_in1 = gds_1
gds_in2 =  gds_2 # 819e-9
gds_out1 = gds_6 # 9628.05e-9
gds_out2 = gds_7  #8693.32e-9

DC_gain = gm_in*gm_out/((gds_in1+gds_in2)*(gds_out1+gds_out2))
print(f"DC gain = {np.log10(DC_gain)*20} dB")

DC gain = 81.09304300417398 dB
